In [1]:
import FluidSim
import numpy as np
from scipy import ndimage
import cv2

def smooth(data, kernel_size = 3):
    kernel = np.ones([kernel_size, kernel_size]) / (kernel_size*kernel_size)
    return ndimage.convolve(data, kernel)

In [9]:
xSize = 200
ySize = 200

walls = np.zeros([xSize, ySize], dtype=bool)
walls[0:2,:]=True
walls[-2:,:]=True

r2=1000
cx = 100
cy = 100
xs = []
ys = []
for x in range(xSize):
    for y in range(ySize):
        r = (x-cx)**2+(y-cy)**2
        if r < r2:
            xs.append(x)
            ys.append(y)
walls[xs, ys] = True

In [11]:
sim = FluidSim.FluidSim(xSize,ySize,2,2, dt=0.00001, walls=walls, nu=0.1)
sim.timestep()

dye = np.zeros_like(sim.u)

upscaleMult = 5
arrowSpacing = 5

imageSize = [sim.nx*upscaleMult, sim.ny * upscaleMult]

upscaledIndices = (sim.indices * upscaleMult)[:,::arrowSpacing,::arrowSpacing]

rowsUpsc = upscaledIndices[1,:,:].flatten()
colsUpsc = upscaledIndices[0,:,:].flatten()

rows = (rowsUpsc / upscaleMult).astype(np.int32)
cols = (colsUpsc / upscaleMult).astype(np.int32)

arrowOrigins = [[rowsUpsc[i], colsUpsc[i]] for i in range(len(rows))]
window_name="PyCFD"

cv2.namedWindow(window_name,cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, imageSize[0],imageSize[1])

fourcc = cv2.VideoWriter_fourcc(*"h264")
video = cv2.VideoWriter('Output.mp4', fourcc,30, [imageSize[0], imageSize[1]], 0)
while True:
    #Simulation steps
    sim.u[:,1] = 1

    sim.timestep()
    dye[90:110,0:20] += 0.5
    dye[xs, ys] = 0

    dye = sim.advectField(dye)
    #dye = smooth(dye, kernel_size=3)

    #Upscale
    scaledImage = cv2.resize(dye/ np.max(dye) * 255, imageSize)
    #scaledImage = cv2.resize(sim.p/ np.max(sim.p) * 255, imageSize)

    scaledImage = (scaledImage ).astype(np.uint8)

    u = (sim.u[rows, cols].flatten()*arrowSpacing*5).astype(np.int32)
    v = (sim.v[rows, cols].flatten()*arrowSpacing*5).astype(np.int32)
    endX = colsUpsc+u
    endY = rowsUpsc+v

    for i in range(len(rows)):
        cv2.arrowedLine(scaledImage, [colsUpsc[i], rowsUpsc[i]], [endX[i], endY[i]], 255)

    
    video.write(scaledImage)
    cv2.imshow(window_name, scaledImage)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video.release()
cv2.destroyAllWindows()
